In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Dropout, GRU, Conv1D, MaxPooling1D, Input, concatenate
from keras.datasets import imdb
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

## 1. Read in data for one store

In [2]:
# read in features of one store (TX_2)
features = pd.read_pickle('TX_2_features.pkl').dropna()

## 2. Define loss function

In [7]:
from keras import backend as K

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

## 3. Set up training and validation sets

In [19]:
NUM_LAG_DAYS= 28

train = features[(features.d<=1863) & (features.d>=1069+NUM_LAG_DAYS)].dropna()
val = features[(features.d > 1863) & (features.d < 1914)].dropna()
test = features[features.d >= 1914]
drop_cols = ['sales','id','d','wm_yr_wk','original_id','store_id','item_id']
num_weeks = NUM_LAG_DAYS // 7
for j in range(1,num_weeks+1):
    drop_cols += ['lag'+str(j) for j in range(j*7-6,j*7)]
drop_cols += ['event_name_1','event_name_2','event_type_1','event_type_2']
x_train = train.drop(drop_cols,axis=1)
y_train = train.sales.values
x_val = val.drop(drop_cols,axis=1)
y_val = val.sales.values    

In [6]:
x_train.shape

(2273370, 48)

## 4. Define training configurations

In [14]:
# training configurations
num_epochs = 100
batch_size = 128
num_nodes = 128
num_layers = 3
dropout = 0.2
loss_fn = rmse
optimizer = 'adam'

## 5. Train MLP model with early stopping

In [15]:
mlp_model = Sequential()
mlp_model.add(Dense(num_nodes, input_dim=x_train.shape[1], activation='relu'))
mlp_model.add(Dropout(dropout))
for i in range(num_layers-1):
    mlp_model.add(Dense(num_nodes, activation='relu'))
    mlp_model.add(Dropout(dropout))
mlp_model.add(Dense(1, activation='relu'))
mlp_model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
# mc = ModelCheckpoint('tfidf_best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
mlp_model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=1)

Train on 2273370 samples, validate on 152450 samples
Epoch 1/100
2273370/2273370 [==============================] - 48s 21us/step - loss: 2.0013 - accuracy: 0.5027 - val_loss: 1.9624 - val_accuracy: 0.4326
Epoch 2/100
2273370/2273370 [==============================] - 46s 20us/step - loss: 1.9304 - accuracy: 0.5022 - val_loss: 1.8894 - val_accuracy: 0.4774
Epoch 3/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9251 - accuracy: 0.5034 - val_loss: 1.6508 - val_accuracy: 0.4174
Epoch 4/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9182 - accuracy: 0.5039 - val_loss: 1.7145 - val_accuracy: 0.3875
Epoch 5/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9186 - accuracy: 0.5053 - val_loss: 1.7206 - val_accuracy: 0.3937
Epoch 6/100
2273370/2273370 [==============================] - 45s 20us/step - loss: 1.9138 - accuracy: 0.5056 - val_loss: 1.6840 - val_accuracy: 0.3987
Epoch 7/100
2273370/2273370 [